In [1]:
# импортируем библиотеки numpy и pandas
import pandas as pd
import numpy as np

# импортируем библиотеку datetime для работы с датами
from datetime import datetime, date

# Задаем некоторые опции библиотеки pandas, которые настраивают вывод
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 10) 
pd.set_option('display.max_rows', 15)
pd.set_option('display.width', 90)

# импортируем библиотеку matplotlib для построения графиков
import matplotlib.pyplot as plt
%matplotlib inline

#### Конкатенация данных

In [2]:
# создаем два объекта Series для конкатенации
s1 = pd.Series(np.arange(0, 3))
s2 = pd.Series(np.arange(5, 8))
s1, s2

(0    0
 1    1
 2    2
 dtype: int64,
 0    5
 1    6
 2    7
 dtype: int64)

In [3]:
# конкатенируем их
pd.concat([s1, s2])

0    0
1    1
2    2
0    5
1    6
2    7
dtype: int64

Вышеприведенный программный код присоединяет метки индекса и значения объекта s2 к конечной метке индекса и конечному значению объекта s1. В результате **получаем дублирующиеся индексные метки**, поскольку в ходе этой операции выравнивание не выполняется.

In [4]:
# создаем два объекта DataFrame для конкатенации, используя те же самые индексные метки
# и имена столбцов, но другие значения
df1 = pd.DataFrame(np.arange(9).reshape(3, 3), columns=['a', 'b', 'c'])
df2 = pd.DataFrame(np.arange(9, 18).reshape(3, 3), columns=['a', 'b', 'c'])
df1, df2

(   a  b  c
 0  0  1  2
 1  3  4  5
 2  6  7  8,
     a   b   c
 0   9  10  11
 1  12  13  14
 2  15  16  17)

In [5]:
# выполняем конкатенацию
# в результате мы получим дублирующиеся индексные метки вдоль индекса строк
pd.concat([df1, df2])

    a   b   c
0   0   1   2
1   3   4   5
2   6   7   8
0   9  10  11
1  12  13  14
2  15  16  17

Следующий программный код демонстрирует выравнивание двух объектов
DataFrame в ходе конкатенации, когда есть общие столбцы (a и c), а также столбцы,
присутствующие только в одном датафрейме (столбец b в датафрейме df1 и столбец d в датафрейме df2).

In [6]:
# демонстрируем конкатенацию двух объектов DataFrame с разными столбцами
df1 = pd.DataFrame(np.arange(9).reshape(3, 3), columns=['a', 'b', 'c'])
df2 = pd.DataFrame(np.arange(9, 18).reshape(3, 3), columns=['a', 'c', 'd'])
df1, df2

(   a  b  c
 0  0  1  2
 1  3  4  5
 2  6  7  8,
     a   c   d
 0   9  10  11
 1  12  13  14
 2  15  16  17)

In [7]:
# выполняем конкатенацию, пропусками будут заполнены значения столбца d в датафрейме df1
# и значения столбца b в датафрейме df2
pd.concat([df1, df2])

    a    b   c     d
0   0  1.0   2   NaN
1   3  4.0   5   NaN
2   6  7.0   8   NaN
0   9  NaN  10  11.0
1  12  NaN  13  14.0
2  15  NaN  16  17.0

Каждой группе данных в итоговом датафрейме можно дать свое название, воспользовавшись параметром **keys**. *Он создает иерархический индекс объекта DataFrame, который позволяет отдельно обращаться к каждой группе данных с помощью свойства .loc объекта DataFrame*. Это удобно, если позже в итоговом объекте DataFrame вам потребуется определить источник получения данных.

In [8]:
# выполняем конкатенацию двух объектов, но при этом создаем индекс с помощью
# заданных ключей
c = pd.concat([df1, df2], keys=['df1', 'df2'])
# обратите внимание на метки строк в выводе
c

        a    b   c     d
df1 0   0  1.0   2   NaN
    1   3  4.0   5   NaN
    2   6  7.0   8   NaN
df2 0   9  NaN  10  11.0
    1  12  NaN  13  14.0
    2  15  NaN  16  17.0

In [9]:
# мы можем извлечь данные, относящиеся к первому или второму исходному датафрейму
c.loc['df2']

    a   b   c     d
0   9 NaN  10  11.0
1  12 NaN  13  14.0
2  15 NaN  16  17.0

#### Переключение осей выравнивания
Функция pd.concat() позволяет задать ось, к которой нужно применить выравнивание во время конкатенации.


In [10]:
# конкатенируем датафреймы df1 и df2 по оси столбцов
# выравниваем по меткам строк, получаем дублирующиеся столбцы
pd.concat([df1, df2], axis='columns')

   a  b  c   a   c   d
0  0  1  2   9  10  11
1  3  4  5  12  13  14
2  6  7  8  15  16  17

В результате получаем дублирующиеся столбцы. Это связано с тем, что в ходе
конкатенации **сначала происходит выравнивание по меткам индекса строк** каждого объекта DataFrame, а затем осуществляется заполнение значений столбцов для первого объекта DataFrame и второго объекта DataFrame независимо от меток индекса строк.

In [11]:
# создаем новый датафрейм d3, чтобы конкатенировать его с датафреймом df1
# датафрейм df3 имеет общую с датафреймом df1 метку 2 и общий столбец a
df3 = pd.DataFrame(np.arange(20, 26).reshape(3, 2),
                  columns=['a', 'd'],
                  index=[2, 3, 4])
df3

    a   d
2  20  21
3  22  23
4  24  25

In [12]:
# конкатерируем их по оси столбцов. Происходит выравнивание по меткам строк, осуществляется
# заполнение значений столбцов df1, а затем столбцов df3, получаем дублирующиеся столбцы
pd.concat([df1, df3], axis='columns')

     a    b    c     a     d
0  0.0  1.0  2.0   NaN   NaN
1  3.0  4.0  5.0   NaN   NaN
2  6.0  7.0  8.0  20.0  21.0
3  NaN  NaN  NaN  22.0  23.0
4  NaN  NaN  NaN  24.0  25.0

Поскольку выравнивание осуществлялось по меткам индекса строк, мы получаем дублирующиеся столбцы. Значения для меток строк 0 и 1 в столбцах a и d, соответствующих датафрейму df3, заполняются пропусками, поскольку эти метки есть только в датафрейме df1. Значения для меток строк 3 и 4 в столбцах a, b и c, соответствующих датафрейму df1, заполняются пропусками, поскольку эти метки
есть только в датафрейме df3.

#### Определение типа соединения

In [13]:
# выполняем внутреннее соединение вместо внешнего
# результат представлен в виде одной строки
pd.concat([df1, df3], axis='columns', join='inner')

   a  b  c   a   d
2  6  7  8  20  21

In [14]:
# добавляем ключи к столбцам
df = pd.concat([df1, df3], keys=['df1', 'df2'], axis='columns', join='inner')
df

  df1       df2    
    a  b  c   a   d
2   6  7  8  20  21

In [15]:
# извлекаем данные из датафрейма с помощью ключа 'df2'
df.loc[:, 'df2']

    a   d
2  20  21

#### Игнорирование меток индекса
Если вы хотите избавиться от дублирования меток в итоговом индексе и при этом
сохранить все строки, вы можете воспользоваться параметром **ignore_index=True**.
Это, по сути, возвращает тот же результат, за исключением того, что теперь наш
индекс имеет тип Int64Index

In [16]:
# избавляемся от дублирования меток в итоговом индексе,
# игнорируя индексные метки в датафреймах-источниках
pd.concat([df1, df2], ignore_index=True)

    a    b   c     d
0   0  1.0   2   NaN
1   3  4.0   5   NaN
2   6  7.0   8   NaN
3   9  NaN  10  11.0
4  12  NaN  13  14.0
5  15  NaN  16  17.0

#### Слияние и соединение данных
Библиотека pandas позволяет выполнить слияние объектов с помощью операций, аналогичных операциям соединения для баз данных, используя функцию **pd.merge()** и метод **.merge()** объекта DataFrame. 

Процедура слияния объединяет данные двух объектов путем поиска совпадающих значений в одном или нескольких индексах столбцов или строк. Затем, применив семантику соединения к этим
значениям, она возвращает новый объект – комбинацию данных обоих объектов.

Слияние полезно, поскольку оно позволяет нам создавать отдельный объект DataFrame для каждого типа данных (одно из правил, позволяющее получить аккуратные данные), а также связывать данные в разных объектах DataFrame с помощью значений, присутствующих в обоих наборах.

#### Слияние данных, расположенных в нескольких объектах
Практический пример использования слияния – это поиск имен и адресов клиентов в заказах.

In [17]:
# это наши клиенты
customers = {'CustomerID': [10, 11],
            'Name': ['Mike', 'Marica'],
            'Address': ['Address for Mike', 'Address for Marcia']}
orders = {'CustomerID': [10, 11, 10],
            'OrderDate': [date(2014, 12, 1), date(2014, 12, 1), date(2014, 12, 1)]
         }

customers = pd.DataFrame(customers)
orders = pd.DataFrame(orders)

customers, '-------', orders

(   CustomerID    Name             Address
 0          10    Mike    Address for Mike
 1          11  Marica  Address for Marcia,
 '-------',
    CustomerID   OrderDate
 0          10  2014-12-01
 1          11  2014-12-01
 2          10  2014-12-01)

In [18]:
# выполняем слияние датафреймов customers и orders так, чтобы мы могли отправить товары
customers.merge(orders)

   CustomerID    Name             Address   OrderDate
0          10    Mike    Address for Mike  2014-12-01
1          10    Mike    Address for Mike  2014-12-01
2          11  Marica  Address for Marcia  2014-12-01

Библиотека pandas совершила для нас какое-то волшебство, выполнив слияние данных с помощью такого простого кода. Она поняла, что у наших датафреймов customers и orders есть общий столбец CustomerID и с этими знаниями. Затем она воспользовалась общими значениями, найденными в столбце CustomerID обоих датафреймов, чтобы связать данные друг с другом и сформировать итоговые данные на основе семантики внутреннего соединения.

In [19]:
# создаем данные, которые будем использовать в качестве примеров
# в оставшейся части этого раздела
left_data = {'key1': ['a', 'b', 'c'],
            'key2': ['x', 'y', 'z'],
            'lval1': [ 0, 1, 2]}

right_data = {'key1': ['a', 'b', 'c'],
            'key2': ['x', 'a', 'z'],
            'rval1': [ 6, 7, 8 ]}
left = pd.DataFrame(left_data)
right = pd.DataFrame(right_data)

left, '-----', right

(  key1 key2  lval1
 0    a    x      0
 1    b    y      1
 2    c    z      2,
 '-----',
   key1 key2  rval1
 0    a    x      6
 1    b    a      7
 2    c    z      8)

In [20]:
# демонстрируем слияние, не указывая столбцы, по которым нужно выполнить слияние
# этот программный код неявно выполняет слияние по всем общим столбцам
left.merge(right)

  key1 key2  lval1  rval1
0    a    x      0      6
1    c    z      2      8

Процедура слияния определяет, что столбцы **key1 и key2 являются общими для обоих объектов DataFrame**. Соответствующие кортежи значений в обоих объектах DataFrame для этих столбцов выглядят как (a, x) и (c, z), поэтому в результате получаем две строки значений.

Чтобы явно задать столбец, используемый для связывания объектов, можно воспользоваться параметром **on**. Следующий программный код выполняет слияние, **используя только значения столбца key1 обоих объектов DataFrame**

In [21]:
# демонстрируем слияние, явно задав столбец, по значениям которого нужно связать
# объекты DataFrame
left.merge(right, on='key1', suffixes=['_l', '_r'])

  key1 key2_l  lval1 key2_r  rval1
0    a      x      0      x      6
1    b      y      1      a      7
2    c      z      2      z      8

In [22]:
# явно выполняем слияние с помощью двух столбцов
left.merge(right, on=['key1', 'key2'])

  key1 key2  lval1  rval1
0    a    x      0      6
1    c    z      2      8

Если вы хотите выполнить слияние на основе столбцов с **разными именами в каждом объекте**, то можете использовать параметры **left_on** и **right_on**, передав имя или имена столбцов каждому соответствующему параметру.

**Т.е мы сами решаем какой столбец брать из левого и правого датафрейма и результатом будут строки, имеющие одинаковые значения.**

In [23]:
left.merge(right, left_on='key1', right_on='key2')

  key1_x key2_x  lval1 key1_y key2_y  rval1
0      a      x      0      b      a      7

Чтобы выполнить слияние с помощью индексных меток строк обоих объектов DataFrame, можно воспользоваться параметрами **left_index=True и right_index=True (нужно задать оба параметра)**

In [24]:
# соединяем индексы строк обеих матриц
pd.merge(left, right, left_index=True, right_index=True, suffixes=['_l', '_r'])

  key1_l key2_l  lval1 key1_r key2_r  rval1
0      a      x      0      a      x      6
1      b      y      1      b      a      7
2      c      z      2      c      z      8

#### Настройка семантики соединения при выполнении слияния
Тип соединения, выполняемый функцией pd.merge() по умолчанию, – внутреннее соединение (**inner join**). Чтобы использовать другой тип соединения, укажите его с помощью параметра **how** функции pd.merge() (или метода .merge()). Допустимыми параметрами являются:

- inner: выполняет пересечение ключей из обоих объектов DataFrame;
- outer: выполняет объединение ключей из обоих объектов DataFrame;
- left: использует только ключи из левого объекта DataFrame;
- right: использует только ключи из правого объекта DataFrame.

In [25]:
# внешнее соединение возращает все строки из внутреннего соединения,
# а также строки датафреймов left и right, не попавшие во внутреннее соединение
# отсутствующие элементы заполняются значениями NaN

In [26]:
left.merge(right, how='outer')

  key1 key2  lval1  rval1
0    a    x    0.0    6.0
1    b    a    NaN    7.0
2    b    y    1.0    NaN
3    c    z    2.0    8.0

Левое соединение возвращает все строки, попавшие во внутреннее соединение (соединенные строки датафреймов left и right, у которых совпали значения в **общих столбцах key1 и key2**), плюс все строки из датафрейма left, не попавшие во внутреннее соединение (для которых не нашлось пары в датафрейме right).

In [27]:
# левое соединение возвращает все строки из внутреннего соединения,
# а также строки датафрейма left, не попавшие во внутреннее соединение
# отсутствующие элементы заполняются значениями NaN
# итоговый датафрейм содержит общие строки датафреймов left и right с метками 0 и 2 (строки
# датафреймов left и right, у которых совпали значения в общих столбцах key1 и key2)
# а также уникальную строку датафрейма left с меткой 1
# уникальная строка датафрейма left в итоговом датафрейме в столбце rval1 заполняется
# значением NaN, потому что в датафрейме left этот столбец отсутствовал
left.merge(right, how='left')

  key1 key2  lval1  rval1
0    a    x      0    6.0
1    b    y      1    NaN
2    c    z      2    8.0

In [28]:
# правое соединение возращает все строки из внутреннего соединения,
# а также строки датафрейма right, не попавшие во внутреннее соединение
# отсутствующие элементы заполняются значениями NaN
# итоговый датафрейм содержит общие строки датафреймов left и right с метками 0 и 1 (строки
# датафреймов left и right, у которых совпали значения в общих столбцах key1 и key2)
# а также уникальную строку датафрейма right с меткой 2
# уникальная строка датафрейма right в итоговом датафрейме в столбце lval1 заполняется
# значением NaN, потому что в датафрейме right этот столбец отсутствовал

right, left.merge(right, how='right')

(  key1 key2  rval1
 0    a    x      6
 1    b    a      7
 2    c    z      8,
   key1 key2  lval1  rval1
 0    a    x    0.0      6
 1    b    a    NaN      7
 2    c    z    2.0      8)

**Кроме того, библиотека pandas предлагает метод .join()**, который можно использовать для **выполнения соединения с помощью <u>индексных меток</u>** двух объектов DataFrame (вместо значений столбцов).

Обратите внимание, что если столбцы в обоих объектах DataFrame не имеют уникальных имен, вы должны указать суффиксы с помощью параметров lsuffix и rsuffix (в отличие от слияния, автоматический суффикс при выполнении этой операции не присваивается). 
